In [1]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD

In [2]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Using <cuda> device


In [3]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [4]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 19

# ====== Dataset ====== #
args.ch_option = {'num_ch': 1,
                  'img_type': 'lwir',
                  'one_ch_option': 'mean'}
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)
args.is_sds = True
args.usages_seg_feats = [True, True, False, False, False, False]


# ====== Optimizer & Training ====== #
args.optim = 'Adam'
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 100
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [ ]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-25',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-55
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [ ]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds, seg_labels) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]
        seg_labels = seg_labels.to(device)

        # Forward prop.
        predicted_locs, predicted_scores, pred_segs = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids,
                         pred_segs, seg_labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

## validation

In [ ]:
# def validation(val_loader, model, criterion):
#     model.eval()

#     num_batches = len(val_loader)
#     losses = AverageMeter()
#     with torch.no_grad():
#         for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
#             images = images.to(device)
#             bboxes = [b.to(device) for b in bboxes]
#             category_ids = [l.to(device) for l in category_ids]

#             predicted_locs, predicted_scores = model(images)
#             loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

#             losses.update(loss, images.size(0))

#     val_loss = losses.avg
#     return val_loss

## checkpoint

In [ ]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [ ]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SDSSSD300(n_classes=args.n_classes,
                                       base=args.base_model,
                                       ch_option=args.ch_option,
                                       usages_seg_feats=args.usages_seg_feats)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.SDSMultiBoxLoss(priors_cxcy=model.priors_cxcy,
                                             usages_seg_feats=model.usages_seg_feats).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [ ]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option,
                                       is_sds=args.is_sds)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.sds_collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [6]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [ ]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10


# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                       model=model,
                       criterion=criterion,
                       optimizer=optimizer)
    run['train/loss'].log(train_loss)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-07-31 13:23:39.037136


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 87.044 (87.044)	Data Time 81.168 (81.168)	Loss 13.6322 (13.6322)	
[68/273]	Batch Time 14.027 (5.623)	Data Time 13.466 (4.992)	Loss 4.5701 (5.8057)	
[136/273]	Batch Time 13.886 (5.047)	Data Time 13.337 (4.456)	Loss 4.6181 (5.2717)	
[204/273]	Batch Time 14.198 (4.853)	Data Time 13.638 (4.275)	Loss 4.4211 (5.0616)	
[272/273]	Batch Time 14.388 (4.757)	Data Time 9.755 (4.171)	Loss 4.4256 (4.9183)	
# ====== Epoch 2 ====== # 2021-07-31 13:45:17.675926
[0/273]	Batch Time 5.115 (5.115)	Data Time 4.482 (4.482)	Loss 4.5527 (4.5527)	
[68/273]	Batch Time 1.989 (1.123)	Data Time 1.337 (0.478)	Loss 4.2942 (4.3308)	
[136/273]	Batch Time 1.910 (1.081)	Data Time 1.306 (0.434)	Loss 4.1445 (4.2646)	
[204/273]	Batch Time 2.142 (1.076)	Data Time 1.528 (0.428)	Loss 3.8118 (4.1913)	
[272/273]	Batch Time 0.474 (1.057)	Data Time 0.000 (0.410)	Loss 3.5543 (4.0513)	
# ====== Epoch 3 ====== # 2021-07-31 13:50:06.339940
[0/273]	Batch Time 5.452 (5.452)	Data Time 4.790 (4.790)	Loss 3.4137 (3.4137)

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.58s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 40.90%
Recall: 0.8243430152143845
# ====== Epoch 11 ====== # 2021-07-31 14:39:32.115150


...


Loading and preparing results...
DONE (t=0.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 25.92%
Recall: 0.8328611898016998
# ====== Epoch 91 ====== # 2021-07-31 21:21:00.898987


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 5.292 (5.292)	Data Time 4.611 (4.611)	Loss 2.1659 (2.1659)	
[68/273]	Batch Time 2.469 (1.124)	Data Time 1.792 (0.471)	Loss 2.3564 (2.2271)	
[136/273]	Batch Time 1.566 (1.092)	Data Time 0.960 (0.439)	Loss 2.1384 (2.2287)	
[204/273]	Batch Time 1.586 (1.086)	Data Time 0.939 (0.432)	Loss 2.0864 (2.2297)	
[272/273]	Batch Time 0.473 (1.072)	Data Time 0.000 (0.421)	Loss 2.1105 (2.2196)	
# ====== Epoch 92 ====== # 2021-07-31 21:25:53.679179
[0/273]	Batch Time 4.736 (4.736)	Data Time 3.963 (3.963)	Loss 2.3088 (2.3088)	
[68/273]	Batch Time 2.695 (1.156)	Data Time 1.930 (0.502)	Loss 1.9622 (2.1916)	
[136/273]	Batch Time 2.245 (1.119)	Data Time 1.601 (0.468)	Loss 2.1948 (2.2080)	
[204/273]	Batch Time 0.619 (1.097)	Data Time 0.005 (0.446)	Loss 2.4396 (2.2123)	
[272/273]	Batch Time 0.475 (1.087)	Data Time 0.000 (0.437)	Loss 2.3092 (2.2216)	
# ====== Epoch 93 ====== # 2021-07-31 21:30:50.550092
[0/273]	Batch Time 4.656 (4.656)	Data Time 3.940 (3.940)	Loss 2.1846 (2.1846)	
[68/273]	

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 26.09%
Recall: 0.8295615275813296


In [7]:
checkpoint = torch.load(checkpoint_path)
save_epoch = checkpoint['epoch']
print(f'Loaded checkpoint from epoch {save_epoch}\n')
model = checkpoint['model']
eval.evaluate(model, ch_option=args.ch_option)

Loaded checkpoint from epoch 100



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.30s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 26.09%
Recall: 0.8295615275813296
